## Imports

In [1]:
# pip install --upgrade scikit-learn

In [2]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.2.4
Sklearn  0.24.1


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [3]:
CLOUD = False

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "your_kaggle_username"
    os.environ['KAGGLE_KEY']      = "your_kaggle_api_key"  # See https://www.kaggle.com/docs/api
    !pip install --upgrade kaggle
    !kaggle competitions download -c titanic
    DATA_PATH = "./"

else:
    DATA_PATH = "../../Datasets/Tabular/titanic/"

## Load data

In [4]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


## Check missings

In [5]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [6]:
df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

# Exercise 1 (2pts):
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [7]:
# CODE HERE get_Title_from_Name funtion
# Create this function using lambda (not def)

get_Title_from_Name = lambda name: name.split(",")[1].strip().split(" ")[0][:-1]


df['Title']      = df['Name'].map(get_Title_from_Name)
df_test['Title'] = df_test['Name'].map(get_Title_from_Name)

# print(df_test["Name"].values[414])
# print(df_test["Title"].values[410:415])
df_test["Title"].value_counts()

Mr        240
Miss       78
Mrs        72
Master     21
Rev         2
Col         2
Ms          1
Dr          1
Dona        1
Name: Title, dtype: int64

In [8]:
assert df['Title'].values[0] == "Mr"
assert df['Title'].values[1] == "Mrs"
assert df['Title'].values[2] == "Miss"

assert df_test['Title'].values[0] == "Mr"
assert df_test['Title'].values[1] == "Mrs"
assert df_test['Title'].values[414] == "Dona"

# Exercise 2 (1pts):
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [9]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

In [10]:
# Use map to apply the prevous dict

df["Title"] =  df["Title"].map(title_dictionary)
df_test["Title"] = df_test["Title"].map(title_dictionary)

df["Title"].isnull().sum()
print(df["Title"].values[886])
print(df_test["Title"].values[417])

Officer
Master


In [11]:
assert df['Title'].values[886] == "Officer"
assert df_test['Title'].values[417] == "Master"

# Exercise OPTINAL (0pts):
Try to extract some information from the feature **Ticket**. Search on Internet if that colum has some kind of information.

In [12]:
#df_test['Ticket']
#get_Title_from_Name = lambda name: name.split(",")[1].strip().split(" ")[0][:-1]
import re
ticket_info = df["Ticket"].str.replace(r'[0-9]+', '0', regex=True)
ticket_info.value_counts()
#df["Ticket"].value_counts()
# From what I have found, the ticket numbers can give some information about who was travelling together,
# useful for identifying families (https://www.kaggle.com/jack89roberts/titanic-using-ticket-groupings)
# Any non-numeric info on the ticket seems to be related to the boarding/leaving locations, interesting but not so useful.

# This shows that most people had unique tickets but there were 134 families, which is something to consider.
df["Ticket"].value_counts().value_counts()

1    547
2     94
3     21
4     11
6      3
7      3
5      2
Name: Ticket, dtype: int64

# Exercise OPTIONAL (0pts):
Try to extract some information from the feature **Cabin**. Search on Internet if that colum has some kind of information.

In [13]:
cabins = df_test['Cabin'].dropna()
counts = []
for cabin in cabins.values:
    counts.append(cabin[0])

pd.Series(counts).value_counts()

# There are a few different cabin types, but majority of passengers were not in a cabin so I'm unsure how helpful this info is
# Especially when we already have df["Pclass"] for determining wealth

C    35
B    18
D    13
E     9
F     8
A     7
G     1
dtype: int64

# Preprocessing
For X data, notice that...
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [14]:
x = df.drop(columns=["Survived", 'Name', 'Ticket', 'Cabin']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket', 'Cabin']) # # X_TEST DATA (NEW DATA)

In [15]:
cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']

Categorical features:
 ['Sex', 'Embarked', 'Title']


# Exercise 3 (2pts):
Create a **ColumnTransformer for Tree Models**. You need to create 2 pipelines (one for numerical and other for categories). Remember:
- Categorical pipeline: Some SimpleImputer -> Some Encoder
- Numerical pipeline: Some SimpleImputer -> NO Encoder

In [16]:
"""
num_preprocessing = pipeline.Pipeline(steps=[
  # Some SimpleImputer here
])

cat_preporcessing = pipeline.Pipeline(steps=[
  # Some SimpleImputer here
  # Some Encoder here. Remember to handle_unknown
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_preprocessing, num_vars),
    ('cat', cat_preporcessing, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro
""";

num_preprocessing = pipeline.Pipeline(steps=[
    # I went with median just so we don't end up with classes/families with fractional values
    ("imputer", impute.SimpleImputer(strategy="median")),
    # As we are using a tree classifier, the rule of thumb is not to do anything to the numeric values
    #("scaler", preprocessing.StandardScaler())
])

cat_preprocessing = pipeline.Pipeline(steps=[
    # By default, imputer uses 0/"missing_value" when filling things in with the constant strategy, I think that's fine for this
    ("imputer", impute.SimpleImputer(strategy="constant")),
    # Rule of thumb for categorical data when using trees is "Ordinal Encoding", where classes become numeric values.
    ("ordinal", preprocessing.OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan)  )
])

# This is a nice pipeline, very concise and organised.
tree_prepro = compose.ColumnTransformer(transformers=[
    # Apply the imputer to our numeric data
    ("num", num_preprocessing, num_vars),
    # Apply the imputer and ordinal encoding to the categorical data
    ("cat", cat_preprocessing, cat_vars),
], remainder="drop") # And finally get rid of everything else

# Rename variables to pass asserts
num_4_treeModels = num_preprocessing
cat_4_treeModels = cat_preprocessing
tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='constant')),
                                                 ('ordinal',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=nan))]),
                                 ['Sex', 'Embarked', 'Title'])])

In [17]:
assert type(tree_prepro)      is compose._column_transformer.ColumnTransformer
assert type(num_4_treeModels) is pipeline.Pipeline
assert type(cat_4_treeModels) is pipeline.Pipeline
assert len(num_4_treeModels) == 1
assert len(cat_4_treeModels) == 2

# Exercise 4 (1pts):
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [18]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [19]:
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(random_state=0),
    # Extra trees needed a bit of tuning to get past 75% accuracy, rest where fine with default settings
  "Extra Trees": ExtraTreesClassifier(random_state=0, n_estimators=1000, bootstrap=True, max_samples=10),
  "Random Forest": RandomForestClassifier(random_state=0),
  "AdaBoost": AdaBoostClassifier(random_state=0),
  "Skl GBM": GradientBoostingClassifier(random_state=0),
  "Skl HistGBM": HistGradientBoostingClassifier(random_state=0),
  "XGBoost": XGBClassifier(random_state=0),
  "LightGBM": LGBMClassifier(random_state=0),
  "CatBoost": CatBoostClassifier(random_state=0),
}


tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

tree_classifiers["CatBoost"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=nan))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('catboostclassifier',
                 <catboost.core.CatBoostClassifier object at 0x7f9d895b6940>)])

In [20]:
for pipe in tree_classifiers.values():
    assert type(pipe) is pipeline.Pipeline

# Exercise 5 (3pts):
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [21]:
"""
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    # CODE HERE
)
"""
x_train, x_val, y_train, y_val = model_selection.train_test_split(x,y, 
                                                                  train_size=0.8,
                                                                  stratify=y,
                                                                  random_state=0)


results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

"""
for model_name, model in tree_classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    
    # GET PREDICTIONS USING x_val
    pred = # CODE HERE

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              
"""
for model_name, model in tree_classifiers.items():
    start_time = time.time()
    model.fit(x_train, y_train)
    pred = model.predict(x_val)
    total_time = time.time() - start_time
    results = results.append({"Model":    model_name,
                          "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                          "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                          "Time":     total_time},
                          ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
# One of these models is spewing out information about it's learning rate, my guess is one of them is "verbose"?
# This is my small-brain fix
clear_output()
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

,Model,Accuracy,Bal Acc.,Time
1,Skl GBM,85.474860,83.050066,0.122452
2,XGBoost,82.681564,80.237154,0.076218
3,Skl HistGBM,81.005587,78.603426,0.383529
4,LightGBM,80.446927,77.608696,0.052636
5,AdaBoost,79.888268,77.964427,0.106724
6,CatBoost,79.888268,77.424242,1.311666
7,Extra Trees,78.770950,76.515152,1.498451
8,Random Forest,78.212291,75.520422,0.196002
9,Decision Tree,77.094972,75.421607,0.022725


In [22]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 6 (3pts):
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [31]:
"""
skf = model_selection.StratifiedKFold(
    # CODE HERE
)
"""
from sklearn.model_selection import cross_val_predict
# This process always uses stratification
skf = model_selection.StratifiedKFold(random_state = 0, n_splits=10, shuffle=True)

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})


# No idea with this, found it frustrating.
"""
for model_name, model in tree_classifiers.items():
    start_time = time.time()
        
    # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y
    pred = # CODE HERE

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              
"""

# for model_name, model in tree_classifiers.items():
#     start_time = time.time()
#     model.fit(x_train, y_train)
#     pred = model.predict(x_val)
    
#     total_time = time.time() - start_time
#     results = results.append({"Model":    model_name,
#                           "Accuracy": metrics.accuracy_score(y_val, pred)*100,
#                           "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
#                           "Time":     total_time},
#                           ignore_index=True)





# results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
# results_ord.index += 1 
# clear_output()
# results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

'\nfor model_name, model in tree_classifiers.items():\n    start_time = time.time()\n        \n    # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y\n    pred = # CODE HERE\n\n    total_time = time.time() - start_time\n\n    results = results.append({"Model":    model_name,\n                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,\n                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,\n                              "Time":     total_time},\n                              ignore_index=True)\n                              \n                              \n'

In [ ]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 7.1
Train with all data the best model

In [34]:
best_model = tree_classifiers["Skl GBM"]

# Fit best model with all data
#best_model.fit(x, y)

#I'm going to assume we don't actually use ALL the data
best_model.fit(x_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=nan))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('gradientboostingclassifier',
                 GradientBoostingClassifier(random_state=0))])

# Exercise 7.2 (2pts)
With your best model, generate the predicitions for test data (x_test)

In [36]:
# test_pred = # Get the predictions for x_test
test_pred = best_model.predict(x_test)

In [37]:
assert len(test_pred) == 418
assert np.unique(test_pred).tolist() == [0,1]

# Exercise 7.3

Submit to kaggle.

- You can use the kaggle command line app. Check https://github.com/Kaggle/kaggle-api

In [38]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [39]:
sub.to_csv("sub.csv")

In [42]:
!kaggle competitions submit -c titanic -f sub.csv -m "I do not understand how to use Stratified kfolds"

100%|██████████████████████████████████████| 2.77k/2.77k [00:02<00:00, 1.17kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster